In [1]:
!pip install "langchain==0.2.11"
!pip install "langchain-community==0.2.6"
!pip install "faiss-cpu"
!pip install "pypdf"
!pip install "sentence-transformers"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 10.0 MB/s  0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
  Attempting uninstall: langchain-text-splitters 0/3 [langchain-core]
    Found existing installation: langchain-text-splitters 0.0.2langchain-core]
    Uninstalling langchain-text-splitters-0.0.2: 0/3 [langchain-core]
      Successfully uninstalled langchain-text-splitters-0.0.2━━━━━ 1/3 [langchain-text-splitters]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain]/3 [langchain]-splitters]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.2.43 which is incompatible.
langchain-classic 1.0.0 requires lang

In [1]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
chemin_pdf = "Societe-Generale-Pilier-3_T2-2022_FR.pdf"
lecteur = PdfReader(chemin_pdf)
texte_brut = ""
for page in lecteur.pages:
    txt = page.extract_text()
    if txt:
        texte_brut += txt
texte_brut

'RAPP O RT  \nSUR LES RIS Q UE S\nĀ\x8e¸\x8eKĄ\x03ј\x03ӳ\x03ј ѕӝѕћӝїѕїї\n2022ABRÉVIATIONS COURANTES\nMillions d’euros : M EUR/Milliards d’euros : Md EUR/ETP : Effectifs en équivalent temps pleinff\nClassements : les sources des classements sont mentionnées explicitement, à défaut, l’information est de source interne.\n\x1c\x18\x16\x16\x0b\x13\x1b\x0f\n\x069\x0f\x14\x15\x12\x12\x1c\x11\x1d\x0f\x16!\x1d\n\n:\x12\r\x0f\x1e\x11\x1f\x1c\x1d\x10\x11\x1c\x15\x1d\x1b\x1f\x11\nTypologiedesrisques\x05\x02\x04 7\nFacteursderisque\x05\x02\x05 8\n\x05\x04;\n\x13\x11\x1d\x1e\x15\x19\x18\x10\x1f\x0f\r\x1a\x15\x1e\r\x16\x11\x1e\r\x10!\x1b\x1f\r\x1e\x15\x19\x18\n\x10\x11\x1d\x12\x19\x18\x10\x1d\x1a\x1c\x19\x1a\x1c\x11\x1d\nChampd’application–Périmètre\x06\x02\x04\nprudentiel 21\nExpositionspondéréesetexigences\x06\x02\x05\nde\xa0fonds\xa0propres 26\nRatioTLAC\x06\x02\x06 27\nRatiodelevier\x06\x02\x07 27\nCoussincontracyclique\x06\x02\x08 28\nInformationsquantitatives\x06\x02\t\ncomplémentairessur\xa0le

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
textes = text_splitter.split_text(texte_brut)

In [4]:
print("🔍 Embeddings avec Ollama…")

embeddings = OllamaEmbeddings(model="all-minilm")

print("Embeddings chargés.")


🔍 Embeddings avec Ollama…
Embeddings chargés.


In [5]:
print("Création de la base vectorielle FAISS…")

docsearch = FAISS.from_texts(textes, embeddings)

print("FAISS OK.")

Création de la base vectorielle FAISS…
FAISS OK.


In [6]:
llm = Ollama(
    model="llama3.2:1b",
    temperature=0.3
)

In [7]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever = docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=False
)

In [8]:
questions = [
    "quel était le montant des RWA à la société générale?",
    "Quels sont les principaux indicateurs de performance financière ?",
    "Qui est l'auteur du document ?",
    "Quels sont les risques mentionnés dans le document ?",
    "Quel est le montant des fonds propres à la fin de la période ?"
]

for q in questions:
    print(f"\n--- Question ---\n{q}")
    rep = chain.invoke(q)
    print(f"\nRéponse : {rep['result']}")


--- Question ---
quel était le montant des RWA à la société générale?

Réponse : En considérant le contexte fourni, il est possible de déterminer que les RWA (Règlements d'assurance vieillesse) à la Société Générale sont liés aux pertes opérationnelles du groupe. Selon l'extrait des chapitres 4.7 et 4.9 du rapport, le risque structurel de change est un élément important pour les exigences en matière d'assurance vieillesse.

En effet, les encours de prêts restructurés et non performants sont mentionnés comme étant une catégorie importante des pertes opérationnelles du groupe. Les provisions et sûretés associées à ces encours peuvent également avoir un impact négatif sur les résultats consolidés.

Il est donc possible de conclure que les RWA à la Société Générale sont liés aux pertes opérationnelles du groupe, mais il est important de noter que les informations fournies ne mentionnent pas explicitement le montant des RWA.

--- Question ---
Quels sont les principaux indicateurs de perfor